In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque
import datetime as dt
import mplfinance 
from renkodf import Renko
from scipy.signal import lfilter
import fx


In [2]:
# collect the actual data from csv file
filename = "C:/Users/WilliamFetzner/Documents/Trading/EURUSD1.csv"
df = pd.read_csv(filename, header=None, names=['date', 'time', 'open', 'high', 'low', 'close', 'volume'])
df.head(10)

,date,time,open,high,low,close,volume
0,1971.01.04,00:00,0.5369,0.5369,0.5369,0.5369,1
1,1971.01.05,00:00,0.5366,0.5366,0.5366,0.5366,1
2,1971.01.06,00:00,0.5365,0.5365,0.5365,0.5365,1
3,1971.01.07,00:00,0.5368,0.5368,0.5368,0.5368,1
4,1971.01.08,00:00,0.5371,0.5371,0.5371,0.5371,1
5,1971.01.11,00:00,0.5371,0.5371,0.5371,0.5371,1
6,1971.01.12,00:00,0.5371,0.5371,0.5371,0.5371,1
7,1971.01.13,00:00,0.5373,0.5373,0.5373,0.5373,1
8,1971.01.14,00:00,0.5372,0.5372,0.5372,0.5372,1
9,1971.01.15,00:00,0.5376,0.5376,0.5376,0.5376,1


In [3]:
# convert date to datetime
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y.%m.%d %H:%M')
# filter the data to just 2023

df_2022 = fx.prep_data(df, 2022)
df_2023 = fx.prep_data(df, 2023)
df_2024 = fx.prep_data(df, 2024)

# Parameters

In [4]:
# Renko variable
initial_brick_size = 0.000325
# create a list of possible brick sizes a max at 0.001 and min at 0.00001 and each step is 0.00001
brick_size_list = np.arange(0.00001, 0.00101, 0.00001)

# psar variables
start = 0.02
increment = 0.02
increment_list = np.arange(0.001, 0.2001, 0.0001)
maximum = 0.2
max_list = np.arange(0.01, 0.501, 0.001)

# impulse variables
lengthMA = 34
ma_list = np.arange(10, 100, 1)
lengthSignal = 9
signal_list = np.arange(1, 51, 1)

# Lot Size
lot_size = 10
lot_sizes_list = np.arange(1, 21, 1)
per_lot = 100000

# Commissions
nova_commission = lot_size * -3
msolutions_commission = lot_size * -5

# starting balance
balance = 200000

# base currency rate
base_currency_rate = 0.045 # EUR
quote_currency_rate = 0.055 # USD

# Optimization

In [5]:
def renko_ready(df, brick_size):
    # create a renko chart from the df dataframe
    r_full = Renko(df, brick_size=brick_size)
    # create a new dataframe from the renko features
    renko_data = r_full.renko_df()
    # rename 'volume' to 'nbars'
    renko_data = renko_data.rename(columns={'volume': 'nbars'})
    return renko_data


In [6]:
def adding_cols(df, inc, max, ma, signal, stop_loss):
    # adding psar to the dataframe
    r_w_psar = fx.psar_from_data(df, inc, max)
    # adding impulse to the dataframe
    r_w_impulse = fx.calc_impulse_macd(r_w_psar, ma, signal)
    # add the brick color to the dataframe
    r_w_impulse['brick_color'] = np.where(r_w_impulse['open'] > r_w_impulse['close'], 'red', 'green')
    # add the day of the week to the dataframe
    r_w_impulse['day_of_week'] = r_w_impulse.index.day_name()
    # place a 1 in day_of_week_transition, if it is the last bar on Friday and the next bar is Sunday
    r_w_impulse['day_of_week_transition'] = np.where((r_w_impulse['day_of_week'] == 'Friday') & 
                                                     ((r_w_impulse['day_of_week'].shift(-1) == 'Sunday') | (r_w_impulse['day_of_week'].shift(-1) == 'Monday')), 1, 0)

    #### entry conditions ####
    # add a column that will be the entry signal for the strategy to be when both impulse_signal and psar_signal are both 'buy' or 'sell'
    r_w_impulse['entry_signal'] = np.where((r_w_impulse['psar_signal'] == 'buy') & (r_w_impulse['impulse_signal'] == 'buy') 
                                            & (r_w_impulse['brick_color'] == 'green') & (r_w_impulse['day_of_week_transition'] != 1), 'long', 
                                            np.where((r_w_impulse['psar_signal'] == 'sell') & (r_w_impulse['impulse_signal'] == 'sell') & 
                                                    (r_w_impulse['brick_color'] == 'red') & (r_w_impulse['day_of_week_transition'] != 1), 'short', 'none'))

    # if there was a change from 'none' to 'buy' or 'sell' then that is an entry signal and replace the 'buy' or 'sell' with 'entry + long' or 'entry + sell'
    r_w_impulse['entry_signal'] = np.where((r_w_impulse['entry_signal'] != 'none') & (r_w_impulse['entry_signal'].shift(1) == 'none'),
                                                'entry + ' + r_w_impulse['entry_signal'], r_w_impulse['entry_signal'])

    #### Exit conditions ####
    # add a stop loss column that will be the entry price +/- the brick size for when the entry signal is 'entry + long' or 'entry + sell'
    r_w_impulse['stop_loss'] = np.where(r_w_impulse['entry_signal'] == 'entry + long', r_w_impulse['open'] - stop_loss,
                                            np.where(r_w_impulse['entry_signal'] == 'entry + short', r_w_impulse['open'] + stop_loss, np.nan))
    
    # if the 'entry_signal' colummn goes from 'entry + short' or 'short' to 'none' then 'none' should be replaced with 'exit' in the entry_signal column
    r_w_impulse['entry_signal'] = np.where((r_w_impulse['entry_signal'].shift(1) == 'entry + short') & (r_w_impulse['entry_signal'] == 'none'), 'exit', 
                                                np.where((r_w_impulse['entry_signal'].shift(1) == 'short') & (r_w_impulse['entry_signal'] == 'none'), 'exit', 
                                                        np.where((r_w_impulse['entry_signal'].shift(1) == 'entry + long') & (r_w_impulse['entry_signal'] == 'none'), 'exit', 
                                                                np.where((r_w_impulse['entry_signal'].shift(1) == 'long') & (r_w_impulse['entry_signal'] == 'none'), 'exit', 
                                                                        r_w_impulse['entry_signal']))))
    # position_count will be a cumulative count used to filter the data to the timeframe between the entry 
    # and exit signals so anytime there is an "entry + long" or "entry + short" the count should increase by 1
    r_w_impulse['position_count'] = np.where(r_w_impulse['entry_signal'] == 'entry + long', 1, np.where(r_w_impulse['entry_signal'] == 'entry + short', 1, 0))
    r_w_impulse['cum_position_count'] = r_w_impulse['position_count'].cumsum()
    # when 'entry_signal' is 'none' then the 'cum_position_count' should be null
    r_w_impulse['cum_position_count'] = np.where(r_w_impulse['entry_signal'] == 'none', np.nan, r_w_impulse['cum_position_count'])
    # group by cum_position_count and forward fill the value in the first index of the 'stop_loss' column
    r_w_impulse['stop_loss'] = r_w_impulse.groupby('cum_position_count')['stop_loss'].ffill()
    # determine whether the exit should be sooner because the stop_loss was hit before the exit signal (look at the high/low of the brick)
    r_w_impulse['exit_stop_loss'] = np.where((r_w_impulse['entry_signal'] == 'long') & (r_w_impulse['stop_loss'] > r_w_impulse['low']), 1,
                                            np.where((r_w_impulse['entry_signal'] == 'short') & (r_w_impulse['stop_loss'] < r_w_impulse['high']), 1, 0))
    
    return r_w_impulse



In [7]:
def calc_profit_loss(df, lots):
    profit_df = pd.DataFrame() #columns=['cum_position_count', 'direction', 'entry_price', 'exit_price', 'first_TP_hit', 'profit']
# calculate the profit for each position by first grouping by each position and finding the entry price
    profit_df['entry_price'] = df.groupby('cum_position_count')['close'].first()
    # separate out the datetime column
    df['datetime'] = df.index
    # get the entry and exit times
    profit_df['entry_time'] = df.groupby('cum_position_count')['datetime'].first()
    profit_df['exit_time'] = df.groupby('cum_position_count')['datetime'].last()
    # # determine the exit price
    profit_df['exit_price'] = df.groupby('cum_position_count')['close'].last()
    # # what was the direction, long or short?
    profit_df['direction'] = df.groupby('cum_position_count')['entry_signal'].first()
    profit_df['direction'] = profit_df['direction'].str.split('+').str[1]
    profit_df['profit'] = np.where((profit_df['direction'].str.strip() == 'long'),
                                            (profit_df['exit_price'] - profit_df['entry_price'])*per_lot*(lots), 
                                            np.where((profit_df['direction'].str.strip() == 'short'),
                                                    (profit_df['entry_price'] - profit_df['exit_price'])*per_lot*(lots), np.nan))
    profit_df = fx.add_swap_rates(profit_df, base_currency_rate, quote_currency_rate, lots=lots)
    profit_df['nova_profit'] = profit_df['profit'] + nova_commission + profit_df['swap_rate']
    profit_df['msolutions_profit'] = profit_df['profit'] + msolutions_commission + profit_df['swap_rate']
    # use the entry time and resample to each day and find the sum of the profit
    profit_df['entry_time'] = pd.to_datetime(profit_df['entry_time'])
    profit_df_new_index = profit_df.set_index('entry_time')
    # find the sum of the nova and msolutions profit for each day
    profit_df_daily = profit_df_new_index.resample('D').agg({'nova_profit': 'sum'})
    # find the number of times that profit_df_daily is below zero
    profit_df_daily['nova_negative'] = np.where(profit_df_daily['nova_profit'] < 0, 1, 0)
    # find the sum of nova_negative and msolutions_negative
    nova_negative_sum = profit_df_daily['nova_negative'].sum()
    # find the sum of nova_profit
    nova_profit_sum = profit_df['nova_profit'].sum()

    return nova_profit_sum, nova_negative_sum

In [13]:
def making_calculations(df, brick_size, inc, max, ma, signal, stop_loss, lot_size):
    renko_data = renko_ready(df, brick_size)
    # add the columns necessary for the strategy
    renko_data_cols_added = adding_cols(renko_data, inc, max, ma, signal, stop_loss)
    # determine the profit/loss for the strategy
    total_profit, days_in_drawdown = calc_profit_loss(renko_data_cols_added, lot_size)

    return total_profit, days_in_drawdown

(20509.999999937732, 15)

In [17]:
# results for standard values
print('2022 data: ', making_calculations(df_2022, initial_brick_size, start, maximum, lengthMA, lengthSignal, initial_brick_size*2, lot_size))
print('2023 data: ', making_calculations(df_2023, initial_brick_size, start, maximum, lengthMA, lengthSignal, initial_brick_size*2, lot_size))
print('2024 data: ', making_calculations(df_2024, initial_brick_size, start, maximum, lengthMA, lengthSignal, initial_brick_size*2, lot_size))
# 2022 data:  ($1,136,669.99, 53)
# 2023 data:  ($447,824.99, 82)
# 2024 data:  ($20,509.99, 15)

2022 data:  (1136669.999999, 53)
2023 data:  (447824.99999925366, 82)
2024 data:  (20509.999999937732, 15)


In [22]:
from tqdm.notebook import tqdm
import statistics

In [23]:
# create an optimization function that goes through all the possible combinations of the variables with tqdm on each loop to show progress
def optimization_fx(df, brick_size_list, increment_list, max_list, ma_list, signal_list, lot_sizes_list):
    # create a list to store the results
    results = []
    # loop through each possible combination of the variables
    for brick_size in tqdm(brick_size_list):
        for inc in tqdm(increment_list):
            for max in tqdm(max_list):
                for ma in tqdm(ma_list):
                    for signal in tqdm(signal_list):
                        for lot_size in tqdm(lot_sizes_list):
                            # calculate the profit/loss for each combination of variables
                            total_profit, days_in_drawdown = making_calculations(df, brick_size, inc, max, ma, signal, brick_size*2, lot_size)
                            # append the results to the results list
                            results.append([brick_size, inc, max, ma, signal, lot_size, total_profit, days_in_drawdown])
    # create a dataframe from the results list
    results_df = pd.DataFrame(results, columns=['brick_size', 'increment', 'maximum', 'ma', 'signal', 'lot_size', 'total_profit', 'days_in_drawdown'])
    # normalize total_profit and days_in_drawdown
    results_df['norm_total_profit'] = statistics.zscore(results_df['total_profit'])
    results_df['norm_days_in_drawdown'] = statistics.zscore(results_df['days_in_drawdown']) * -1
    # find the total of the normalized columns
    results_df['score'] = results_df['norm_total_profit'] + results_df['norm_days_in_drawdown']

    # find the best results
    most_profitable = results_df[results_df['total_profit'] == results_df['total_profit'].max()]
    least_drawdown = results_df[results_df['days_in_drawdown'] == results_df['days_in_drawdown'].min()]
    best_results = results_df[results_df['score'] == results_df['score'].max()]
    return results_df, most_profitable, least_drawdown, best_results

In [24]:
results_df, most_profitable, least_drawdown, best_results = optimization_fx(df_2022, brick_size_list, increment_list, max_list, ma_list, signal_list, lot_sizes_list)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1991 [00:00<?, ?it/s]

  0%|          | 0/491 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
best_results

In [ ]:
least_drawdown

In [ ]:
most_profitable